# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import zipfile
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
orders = pd.read_csv('orders.zip',compression='zip')

In [3]:
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [4]:
orders.shape

(397924, 14)

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [5]:
# your code here
# Sub Problem 1: We use and order by for the customers and aggregate the amount spent by sum.
customers = orders.groupby(['CustomerID']).agg({'amount_spent':sum})
display(customers.head(5))

,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40


In [6]:
customers.shape

(4339, 1)

In [7]:
# Sub Problem 2: We bin the customers into 20 bins.
# So each one represents a 5% step and we can select the customers from our desired bins.
# We use qcut so the customers and not the amount spent is evenly distributed
cus_bin = pd.qcut(customers['amount_spent'],20, labels = list(range(1,21)))
display(cus_bin.head(3))

CustomerID
12346    20
12347    19
12348    16
Name: amount_spent, dtype: category
Categories (20, int64): [1 < 2 < 3 < 4 ... 17 < 18 < 19 < 20]

In [8]:
# Sub Problem 3: We create a new column 'customer_status' where we enter the vip-status of the customer depending on his quantile
# We have to translate the bins to actual descriptions.
# Then we have to assign the descriptions to the customers in the orders dataframe according to ther customer id
# We create a 'Regular' label for all other customers to not mix in NaN or number values
def status_sort(bin):
    if bin == 20:
        return 'VIP'
    elif bin > 15:
        return 'Preferred'
    else:
        return 'Regular'

In [9]:
# Now we assign the bins with the descriptions

cus_bin = pd.DataFrame(cus_bin)

In [10]:
cus_bin['status'] = cus_bin['amount_spent'].apply(status_sort)

In [11]:
# We only leave 'status' and 'CustomerID' to simplify the merge, also we turn the index into a regular column
cus_bin = cus_bin[['status']]
cus_bin.head()

,status
CustomerID,
12346,VIP
12347,Preferred
12348,Preferred
12349,Preferred
12350,Regular


In [12]:
cus_bin = cus_bin.reset_index()
cus_bin.head()

,CustomerID,status
0,12346,VIP
1,12347,Preferred
2,12348,Preferred
3,12349,Preferred
4,12350,Regular


In [13]:
# Now we transfer the status to the orders table with a merge
orders = pd.merge(left = orders,
                                 right = cus_bin,
                                 how = 'left', 
                                 left_on = "CustomerID", 
                                 right_on= "CustomerID")

In [14]:
# Check result
orders.head(5)

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,status
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,Preferred
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,Preferred
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

First we group the orders table by country and count the occurances of VIP customers

Afterwards we can select the country with the most occurances

In [15]:
# your code here
countries = orders.groupby(['Country','status']).agg(status_count=('status','count')).reset_index()

In [16]:
# Check result
countries = pd.DataFrame(countries)
display(countries)

,Country,status,status_count
0,Australia,Preferred,130
1,Australia,Regular,157
2,Australia,VIP,898
3,Austria,Preferred,158
4,Austria,Regular,240
...,...,...,...
71,United Arab Emirates,Regular,68
72,United Kingdom,Preferred,137450
73,United Kingdom,Regular,132710
74,United Kingdom,VIP,84185


In [17]:
# Now we can select the rows with the VIP count
vip_max = countries[countries['status'] == 'VIP']
display(vip_max.head(5))

,Country,status,status_count
2,Australia,VIP,898
8,Belgium,VIP,54
14,Channel Islands,VIP,364
17,Cyprus,VIP,248
21,Denmark,VIP,36


In [18]:
# From here we select the highest and can see which country it is
vip_max = vip_max[vip_max['status_count'] == vip_max['status_count'].max()]
display(vip_max.head(5))

,Country,status,status_count
74,United Kingdom,VIP,84185


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

We create a dataframe where we drop all rows for regular customers.
Afterwards we can regroup by countries, ignoring the status, that gives us the overall count for VIP and Preferred.
From there we select the row with the highest count.

In [19]:
# Dropping the rows of regular customers:

In [20]:
countries = countries[~countries['status'].isin(['Regular'])]
countries.head(10)

,Country,status,status_count
0,Australia,Preferred,130
2,Australia,VIP,898
3,Austria,Preferred,158
6,Belgium,Preferred,1503
8,Belgium,VIP,54
10,Canada,Preferred,135
12,Channel Islands,Preferred,225
14,Channel Islands,VIP,364
15,Cyprus,Preferred,203
17,Cyprus,VIP,248


In [21]:
# Regroup by countries, ignore status, this time we have to sum up the already counted statuses
countries2 = countries.groupby(['Country']).agg(status_count=('status_count','sum')).reset_index()
countries2.head(10)

,Country,status_count
0,Australia,1028
1,Austria,158
2,Belgium,1557
3,Canada,135
4,Channel Islands,589
5,Cyprus,451
6,Denmark,217
7,EIRE,7238
8,Finland,504
9,France,6301


In [22]:
# Now we choose the row with the highest count
cumax = countries2[countries2['status_count'] == countries2['status_count'].max()]
display(cumax)

,Country,status_count
26,United Kingdom,221635
